In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
def make_mini_batch(train_data, size_of_mini_batch, length_of_sequences):
    inputs  = np.empty(0)
    outputs = np.empty(0)
    for _ in range(size_of_mini_batch):
        index   = random.randint(0, len(train_data) - length_of_sequences)
        part    = train_data[index:index + length_of_sequences]
        inputs  = np.append(inputs, part[:, 0])
        outputs = np.append(outputs, part[-1, 1])
    inputs  = inputs.reshape(-1, length_of_sequences, 1)
    outputs = outputs.reshape(-1, 1)
    return (inputs, outputs)

In [3]:
def make_prediction_initial(train_data, index, length_of_sequences):
    return train_data[index:index + length_of_sequences, 0]

In [4]:
train_data_path             = "./data.npy"
num_of_input_nodes          = 1
num_of_hidden_nodes         = 2
num_of_output_nodes         = 1
length_of_sequences         = 30
num_of_training_epochs      = 800
length_of_initial_sequences = 30
num_of_prediction_epochs    = 100
size_of_mini_batch          = 2
learning_rate               = 0.1
forget_bias                 = 1.0

In [5]:
train_data = np.load(train_data_path)
print("train_data:", train_data)

('train_data:', array([[-81.        ,  30.        ],
       [-81.        ,  29.83333333],
       [-80.        ,  29.66666667],
       [-78.        ,  29.5       ],
       [-75.        ,  29.33333333],
       [-73.        ,  29.16666667],
       [-71.        ,  29.        ],
       [-70.        ,  28.83333333],
       [-69.        ,  28.66666667],
       [-67.        ,  28.5       ],
       [-68.        ,  28.33333333],
       [-69.        ,  28.16666667],
       [-69.        ,  28.        ],
       [-69.        ,  27.83333333],
       [-69.        ,  27.66666667],
       [-70.        ,  27.5       ],
       [-71.        ,  27.33333333],
       [-70.        ,  27.16666667],
       [-71.        ,  27.        ],
       [-72.        ,  26.83333333],
       [-72.        ,  26.66666667],
       [-72.        ,  26.5       ],
       [-71.        ,  26.33333333],
       [-71.        ,  26.16666667],
       [-71.        ,  26.        ],
       [-70.        ,  25.83333333],
       [-69.        , 

In [6]:
# random seed fix
random.seed(0)
np.random.seed(0)
tf.set_random_seed(0)

In [7]:
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)

In [8]:
with tf.Graph().as_default():
    input_ph      = tf.placeholder(tf.float32, [None, length_of_sequences, num_of_input_nodes], name="input")
    supervisor_ph = tf.placeholder(tf.float32, [None, num_of_output_nodes], name="supervisor")
    istate_ph     = tf.placeholder(tf.float32, [None, num_of_hidden_nodes * 2], name="istate")

    with tf.name_scope("inference") as scope:
        weight1_var = tf.Variable(tf.truncated_normal([num_of_input_nodes, num_of_hidden_nodes], stddev=0.1), name="weight1")
        weight2_var = tf.Variable(tf.truncated_normal([num_of_hidden_nodes, num_of_output_nodes], stddev=0.1), name="weight2")
        bias1_var   = tf.Variable(tf.truncated_normal([num_of_hidden_nodes], stddev=0.1), name="bias1")
        bias2_var   = tf.Variable(tf.truncated_normal([num_of_output_nodes], stddev=0.1), name="bias2")

        in1 = tf.transpose(input_ph, [1, 0, 2])         # (batch, sequence, data) -> (sequence, batch, data)
        in2 = tf.reshape(in1, [-1, num_of_input_nodes]) # (sequence, batch, data) -> (sequence * batch, data)
        in3 = tf.matmul(in2, weight1_var) + bias1_var
        in4 = tf.split(0, length_of_sequences, in3)     # sequence * (batch, data)

        cell = tf.nn.rnn_cell.BasicLSTMCell(num_of_hidden_nodes, forget_bias=forget_bias)
        rnn_output, states_op = tf.nn.rnn(cell, in4, initial_state=istate_ph)
        output_op = tf.matmul(rnn_output[-1], weight2_var) + bias2_var

    with tf.name_scope("loss") as scope:
        square_error = tf.reduce_mean(tf.square(output_op - supervisor_ph))
        loss_op      = square_error
        tf.scalar_summary("loss", loss_op)

    with tf.name_scope("training") as scope:
        training_op = optimizer.minimize(loss_op)

    summary_op = tf.merge_all_summaries()
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        saver = tf.train.Saver()
        summary_writer = tf.train.SummaryWriter("data", graph=sess.graph)
        sess.run(init)

        for epoch in range(num_of_training_epochs):
            inputs, supervisors = make_mini_batch(train_data, size_of_mini_batch, length_of_sequences)

            train_dict = {
                input_ph:      inputs,
                supervisor_ph: supervisors,
                istate_ph:     np.zeros((size_of_mini_batch, num_of_hidden_nodes * 2)),
            }
            sess.run(training_op, feed_dict=train_dict)

            if (epoch + 1) % 10 == 0:
                summary_str, train_loss = sess.run([summary_op, loss_op], feed_dict=train_dict)
                summary_writer.add_summary(summary_str, epoch)
                print("train#%d, train loss: %e" % (epoch + 1, train_loss))

        outputs = np.empty(0)
        states  = np.zeros((num_of_hidden_nodes * 2)),

        for epoch in range(num_of_prediction_epochs):
            inputs  = make_prediction_initial(train_data, epoch, length_of_initial_sequences)

            pred_dict = {
                input_ph:  inputs.reshape((1, length_of_sequences, 1)),
                istate_ph: states,
            }
            output, states = sess.run([output_op, states_op], feed_dict=pred_dict)
            print("prediction#%d, output: %f" % (epoch + 1, output))

            #inputs  = np.delete(inputs, 0)
            #inputs  = np.append(inputs, output)
            outputs = np.append(outputs, output)

        print("outputs:", outputs)
        np.save("output.npy", outputs)

        saver.save(sess, "data/model")

train#10, train loss: 9.042835e+00
train#20, train loss: 2.968634e+01
train#30, train loss: 6.229526e+01
train#40, train loss: 1.726133e+00
train#50, train loss: 8.695898e+01
train#60, train loss: 5.000567e+01
train#70, train loss: 2.533859e+02
train#80, train loss: 2.213429e+02
train#90, train loss: 8.634124e+01
train#100, train loss: 2.267053e+02
train#110, train loss: 1.057280e+01
train#120, train loss: 1.596899e+02
train#130, train loss: 1.780573e+01
train#140, train loss: 1.241941e+02
train#150, train loss: 9.575169e+01
train#160, train loss: 7.781107e+01
train#170, train loss: 8.869485e+01
train#180, train loss: 1.845793e+00
train#190, train loss: 1.516957e+02
train#200, train loss: 1.679357e+01
train#210, train loss: 2.409385e+02
train#220, train loss: 5.681495e+01
train#230, train loss: 1.214334e+02
train#240, train loss: 4.997749e+01
train#250, train loss: 2.308288e+02
train#260, train loss: 2.102325e+02
train#270, train loss: 9.687025e+01
train#280, train loss: 8.724185e+01
t